In [ ]:
#! pip install -U textblob
#! python -m textblob.download_corpora

In [1]:
import pandas as pd
import re 
import seaborn as sns
import numpy as np
from textblob import TextBlob

In [2]:
tw_clean = pd.read_csv("tw_data_short.csv")
tw_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567526 entries, 0 to 567525
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            567526 non-null  object 
 1   time            567526 non-null  object 
 2   date_final      567526 non-null  object 
 3   tweet           567526 non-null  object 
 4   mentions        567526 non-null  object 
 5   replies_count   567526 non-null  float64
 6   retweets_count  567526 non-null  float64
 7   likes_count     567526 non-null  float64
 8   hashtags        567526 non-null  object 
dtypes: float64(3), object(6)
memory usage: 39.0+ MB


In [3]:
tw_tokens = pd.read_csv("tweets_token.csv")
tw_tokens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567526 entries, 0 to 567525
Columns: 198 entries, Unnamed: 0 to gt
dtypes: int64(198)
memory usage: 857.3 MB


In [4]:
all_data_token = tw_clean.join(tw_tokens, how='left', lsuffix='l', rsuffix='r', sort=False )
all_data_token["date"] = pd.to_datetime(all_data_token["date"])
all_data_token["timel"] = pd.to_datetime(all_data_token["timel"], format= '%H:%M:%S' ).dt.time
all_data_token["date_final"] = pd.to_datetime(all_data_token["date_final"])
all_data_token.rename(columns={"date_final": "time"}, inplace=True)
all_data_token

,date,timel,time,tweetl,mentions,replies_count,retweets_count,likes_count,hashtags,Unnamed: 0,...,giving,cap,l,usdt,year,atom,luna,ladybug,already,gt
0,2021-03-13,15:59:55,2021-03-14 00:59:55,Where's $Doge going?,[],0.0,1.0,1.0,[],0,...,0,0,0,0,0,0,0,0,0,0
1,2021-03-13,15:59:53,2021-03-14 00:59:53,Gonna start accepting dogecoin as payment boii...,[],3.0,3.0,10.0,"['doge', 'dogearmy', 'dogecoins', 'dogecoinris...",1,...,0,0,0,0,0,0,0,0,0,0
2,2021-03-13,15:59:52,2021-03-14 00:59:52,Buy and Hold #Dogecoins #doge #dogearmy #Dogec...,[],3.0,14.0,39.0,"['dogecoins', 'doge', 'dogearmy', 'dogecoinris...",2,...,0,0,0,0,0,0,0,0,0,0
3,2021-03-13,15:59:38,2021-03-14 00:59:38,"@elonmusk Forget btc 80% gain, forget 10x doge...","[{'screen_name': 'digicolofficial', 'name': 'd...",4.0,1.0,7.0,[],3,...,0,0,0,0,0,0,0,0,0,0
4,2021-03-13,15:59:34,2021-03-14 00:59:34,"“Oh. I remember you. How's it going, little Do...",[],0.0,1.0,6.0,[],4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567521,2021-03-28,17:02:01,2021-03-29 02:02:01,Mar 27 Issue: “Theta Network: Theta…” https:/...,[],2.0,5.0,39.0,"['blockchain', 'bitcoin', 'defi', 'nft', 'stak...",567521,...,0,0,0,0,0,0,0,0,0,0
567522,2021-03-28,17:01:18,2021-03-29 02:01:18,1. Retweet + Like #CTBot Adrop Ram 0.1 DOGE 10,[],0.0,9.0,9.0,['ctbot'],567522,...,0,0,0,0,0,0,0,0,0,0
567523,2021-03-28,17:01:15,2021-03-29 02:01:15,🔉🔉🔊🔉 #dogeparty I can’t hear YOU!! Let’s make ...,[],1.0,2.0,9.0,"['dogeparty', 'doge', 'dogecoin', 'dogecointot...",567523,...,0,0,0,0,0,0,0,0,0,0
567524,2021-03-28,17:00:14,2021-03-29 02:00:14,🔉🔉🔊🔉 #dogeparty I can’t hear YOU!! Let’s make ...,[],1.0,1.0,6.0,"['dogeparty', 'doge', 'dogecoin', 'dogecointot...",567524,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
testimonial = TextBlob(tw_clean.tweet[12344])
print(testimonial.sentiment.subjectivity)
print(testimonial.sentiment.polarity)

In [5]:
all_data_token.drop(labels = ["hashtags", "Unnamed: 0"], axis=1, inplace=True)

In [6]:
#Adding col with 0 1 values, wich mean more or less 100 likes
all_data_token.insert(7,'top_tw', '')
all_data_token['top_tw'] = pd.DataFrame(np.where(all_data_token['likes_count']>100, '1', '0'))
all_data_token['top_tw'].value_counts()

0    545461
1     22065
Name: top_tw, dtype: int64

In [7]:
all_data_token["top_tw"] = all_data_token["top_tw"].apply(pd.to_numeric)

In [26]:
all_data_token.head()

,date,timel,time,tweetl,mentions,replies_count,retweets_count,top_tw,likes_count,dogecoin,...,giving,cap,l,usdt,year,atom,luna,ladybug,already,gt
0,2021-03-13,15:59:55,2021-03-14 00:59:55,Where's $Doge going?,[],0.0,1.0,0,1.0,0,...,0,0,0,0,0,0,0,0,0,0
1,2021-03-13,15:59:53,2021-03-14 00:59:53,Gonna start accepting dogecoin as payment boii...,[],3.0,3.0,0,10.0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021-03-13,15:59:52,2021-03-14 00:59:52,Buy and Hold #Dogecoins #doge #dogearmy #Dogec...,[],3.0,14.0,0,39.0,0,...,0,0,0,0,0,0,0,0,0,0
3,2021-03-13,15:59:38,2021-03-14 00:59:38,"@elonmusk Forget btc 80% gain, forget 10x doge...","[{'screen_name': 'digicolofficial', 'name': 'd...",4.0,1.0,0,7.0,0,...,0,0,0,0,0,0,0,0,0,0
4,2021-03-13,15:59:34,2021-03-14 00:59:34,"“Oh. I remember you. How's it going, little Do...",[],0.0,1.0,0,6.0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#splitting to popular and unpopular tweets
top_tw = all_data_token[all_data_token['top_tw'] == 1]
low_tw = all_data_token[all_data_token['top_tw'] == 0]
top_tw.head()

,date,timel,time,tweetl,mentions,replies_count,retweets_count,top_tw,likes_count,dogecoin,...,giving,cap,l,usdt,year,atom,luna,ladybug,already,gt
18,2021-03-13,15:58:13,2021-03-14 00:58:13,doge elon bots are alive and well 👍 https://t...,[],6.0,3.0,1,104.0,0,...,0,0,0,0,0,0,0,0,0,0
30,2021-03-13,15:57:00,2021-03-14 00:57:00,$DOGE Let’s See $0.10 By Tonight? #DogecoinRis...,[],19.0,94.0,1,390.0,1,...,0,0,0,0,0,0,0,0,0,0
35,2021-03-13,15:56:22,2021-03-14 00:56:22,@elonmusk Is it $doge or $dego You shouldnt w...,[],26.0,2.0,1,121.0,2,...,0,0,0,0,0,0,0,0,0,0
43,2021-03-13,15:55:58,2021-03-14 00:55:58,Cmon $DOGE Let’s See $0.07 By Tonight. #Dogeco...,[],59.0,535.0,1,2350.0,0,...,0,0,0,0,0,0,0,0,0,0
69,2021-03-13,15:54:01,2021-03-14 00:54:01,@elonmusk You + $DOGE make our day always,[],4.0,22.0,1,128.0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
all_data_token.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567526 entries, 0 to 567525
Columns: 206 entries, date to gt
dtypes: datetime64[ns](2), float64(3), int64(198), object(3)
memory usage: 892.0+ MB


In [11]:
dict_cols = {i : "sum" for i in all_data_token.columns[5:-1]}
dict_cols

{'replies_count': 'sum',
 'retweets_count': 'sum',
 'top_tw': 'sum',
 'likes_count': 'sum',
 'dogecoin': 'sum',
 'dogearmy': 'sum',
 'elonmusk': 'sum',
 'buy': 'sum',
 'dogecointothemoon': 'sum',
 'btc': 'sum',
 'crypto': 'sum',
 'dogecoinrise': 'sum',
 'dogecoinarmy': 'sum',
 'eth': 'sum',
 'get': 'sum',
 'bitcoin': 'sum',
 'moon': 'sum',
 'dollar': 'sum',
 'coin': 'sum',
 'one': 'sum',
 'like': 'sum',
 'xrp': 'sum',
 'let': 'sum',
 'go': 'sum',
 'hold': 'sum',
 'k': 'sum',
 'people': 'sum',
 'dogecoins': 'sum',
 'going': 'sum',
 'cryptocurrency': 'sum',
 'dogecointo': 'sum',
 'dogetothemoon': 'sum',
 'price': 'sum',
 'dogeday': 'sum',
 'ada': 'sum',
 'today': 'sum',
 'dogecointoadollar': 'sum',
 'timer': 'sum',
 'bnb': 'sum',
 'link': 'sum',
 'amp': 'sum',
 'hodl': 'sum',
 'make': 'sum',
 'last': 'sum',
 'day': 'sum',
 'pump': 'sum',
 'sell': 'sum',
 'money': 'sum',
 'see': 'sum',
 'elon': 'sum',
 'new': 'sum',
 'retweet': 'sum',
 'ltc': 'sum',
 'amc': 'sum',
 'good': 'sum',
 'u': 's

In [12]:
#grouping by date and hour
top_tw_agg = top_tw.groupby([top_tw['time'].dt.date, top_tw['time'].dt.hour]).agg(dict_cols)
low_tw_agg = low_tw.groupby([low_tw['time'].dt.date, low_tw['time'].dt.hour]).agg(dict_cols)

In [13]:
top_tw_agg

replies_count  retweets_count  top_tw  likes_count  dogecoin  \
time       time                                                                 
2021-01-01 18             82.0            72.0       2        562.0         3   
           19              8.0            15.0       1        111.0         0   
           20             29.0            39.0       1        281.0         1   
           21             64.0           146.0       4       1417.0         3   
           22             14.0            23.0       1        166.0         1   
...                        ...             ...     ...          ...       ...   
2021-04-29 4              78.0           431.0       7       2989.0         4   
           5             499.0           766.0       5       1751.0         0   
           6              57.0           316.0       3        871.0         1   
           7            1174.0          2506.0       5      11779.0         3   
           8            1471.0          4502.0      40      26748.0        25   

                 dogearmy  elonmusk  buy  dogecointothemoon  btc  ...  rise  \
time       time                                                   ...         
2021-01-01 18           2         0    1                  1    0  ...     0   
           19           0         1    0                  0    1  ...     0   
           20           0         0    0                  0    1  ...     0   
           21           1         0    1                  2    0  ...     0   
           22           0         1    1                  1    1  ...     0   
...                   ...       ...  ...                ...  ...  ...   ...   
2021-04-29 4            4         0    1                  1    0  ...     0   
           5            1         0    1                  0    1  ...     0   
           6            2         0    0                  1    0  ...     0   
           7            4         1    1                  1    0  ...     1   
           8           21        15    3                  6    4  ...     0   

                 giving  cap  l  usdt  year  atom  luna  ladybug  already  
time       time                                                            
2021-01-01 18         0    0  0     0     0     0     0        0        0  
           19         0    0  0     0     0     0     0        0        0  
           20         0    0  0     0     0     0     0        0        0  
           21         0    0  0     0     0     0     0        0        0  
           22         0    0  0     0     0     0     0        0        0  
...                 ...  ... ..   ...   ...   ...   ...      ...      ...  
2021-04-29 4          0    0  0     0     0     0     0        0        0  
           5          0    0  0     0     0     0     0        0        0  
           6          0    0  0     0     0     0     0        0        0  
           7          0    0  0     0     0     0     0        0        0  
           8          0    0  0     0     0     0     0        0        0  

[2370 rows x 200 columns]

In [14]:
data_prices = pd.read_csv("data/dataprices.csv")

In [15]:
data_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16875 entries, 0 to 16874
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    16875 non-null  object 
 1   open    16875 non-null  float64
 2   high    16875 non-null  float64
 3   low     16875 non-null  float64
 4   Volume  16875 non-null  float64
 5   tv      16875 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 791.1+ KB


In [16]:
data_prices['time'] = pd.to_datetime(data_prices['time'])
data_prices.head()

,time,open,high,low,Volume,tv
0,2021-01-01 00:00:00,0.004683,0.004702,0.004616,1865676.0,1
1,2021-01-01 01:00:00,0.004705,0.004766,0.004701,1082226.0,1
2,2021-01-01 02:00:00,0.004733,0.004765,0.004733,85161.0,2
3,2021-01-01 03:00:00,0.004754,0.005001,0.004754,5030766.0,5
4,2021-01-01 04:00:00,0.005000,0.005001,0.004816,5531754.0,-4


In [17]:
data_prices = data_prices.loc[(data_prices["time"] > "2021-01-02 00:00:00") & (data_prices["time"] < "2021-04-29 01:00:00")]

In [18]:
data_prices

,time,open,high,low,Volume,tv
25,2021-01-02 01:00:00,0.005818,0.005854,0.005564,11123815.0,-3
26,2021-01-02 02:00:00,0.005754,0.006431,0.005701,32955177.0,5
27,2021-01-02 03:00:00,0.006353,0.006791,0.006066,108807546.0,5
28,2021-01-02 04:00:00,0.006670,0.007981,0.006654,166272032.0,5
29,2021-01-02 05:00:00,0.007756,0.007776,0.006965,90792179.0,-5
...,...,...,...,...,...,...
2820,2021-04-28 20:00:00,0.310310,0.310760,0.301330,19680380.6,-3
2821,2021-04-28 21:00:00,0.306520,0.315320,0.304800,20889588.2,3
2822,2021-04-28 22:00:00,0.310590,0.326530,0.308500,32387298.2,5
2823,2021-04-28 23:00:00,0.321110,0.326890,0.316570,33777982.1,2


In [19]:
data_prices = data_prices.groupby([data_prices['time'].dt.date, data_prices['time'].dt.hour]).agg({"open": "sum", "high": "sum", "low": "sum", "tv": "sum", "Volume": "sum"})


In [20]:
pd.DataFrame(data_prices)

open      high       low  tv       Volume
time       time                                               
2021-01-02 1     0.005818  0.005854  0.005564  -3   11123815.0
           2     0.005754  0.006431  0.005701   5   32955177.0
           3     0.006353  0.006791  0.006066   5  108807546.0
           4     0.006670  0.007981  0.006654   5  166272032.0
           5     0.007756  0.007776  0.006965  -5   90792179.0
...                   ...       ...       ...  ..          ...
2021-04-28 20    0.310310  0.310760  0.301330  -3   19680380.6
           21    0.306520  0.315320  0.304800   3   20889588.2
           22    0.310590  0.326530  0.308500   5   32387298.2
           23    0.321110  0.326890  0.316570   2   33777982.1
2021-04-29 0     0.323320  0.323320  0.306930  -5   34687125.8

[2800 rows x 5 columns]

In [21]:
data_prices.describe()

,open,high,low,tv,Volume
count,2800.000000,2800.000000,2800.000000,2800.000000,2.800000e+03
mean,0.072687,0.074621,0.070898,0.024286,2.668762e+07
std,0.083215,0.086436,0.080163,3.120559,5.713087e+07
min,0.005754,0.005854,0.005564,-5.000000,9.028900e+04
25%,0.032592,0.033352,0.031404,-2.000000,3.230013e+06
50%,0.054496,0.055097,0.054081,0.000000,7.743748e+06
75%,0.060882,0.061654,0.059733,2.000000,2.486749e+07
max,0.425267,0.480000,0.404310,5.000000,9.334161e+08


In [22]:
#merging data with pricess
tokens_m = top_tw_agg.merge(low_tw_agg, how = "outer",suffixes=('_t', '_l'), left_index = True, right_index = True)


In [23]:
#merging data with pricess
all_data_m = tokens_m.merge(data_prices, how = "inner", left_index = True, right_index = True)
all_data_m.fillna(0, inplace=True)

In [24]:
all_data_m.to_csv("data/all_data_m.csv")

In [25]:
all_data_m.head()

replies_count_t  retweets_count_t  top_tw_t  likes_count_t  \
time       time                                                               
2021-01-02 1                 0.0               0.0       0.0            0.0   
           2                 0.0               0.0       0.0            0.0   
           3                 0.0               0.0       0.0            0.0   
           4                 0.0               0.0       0.0            0.0   
           5                 0.0               0.0       0.0            0.0   

                 dogecoin_t  dogearmy_t  elonmusk_t  buy_t  \
time       time                                              
2021-01-02 1            0.0         0.0         0.0    0.0   
           2            0.0         0.0         0.0    0.0   
           3            0.0         0.0         0.0    0.0   
           4            0.0         0.0         0.0    0.0   
           5            0.0         0.0         0.0    0.0   

                 dogecointothemoon_t  btc_t  ...  year_l  atom_l  luna_l  \
time       time                              ...                           
2021-01-02 1                     0.0    0.0  ...       0       0       0   
           2                     0.0    0.0  ...       0       1       0   
           3                     0.0    0.0  ...       0       0       0   
           4                     0.0    0.0  ...       0       0       0   
           5                     0.0    0.0  ...       1       1       1   

                 ladybug_l  already_l      open      high       low  tv  \
time       time                                                           
2021-01-02 1             0          0  0.005818  0.005854  0.005564  -3   
           2             0          0  0.005754  0.006431  0.005701   5   
           3             0          0  0.006353  0.006791  0.006066   5   
           4             0          0  0.006670  0.007981  0.006654   5   
           5             0          1  0.007756  0.007776  0.006965  -5   

                      Volume  
time       time               
2021-01-02 1      11123815.0  
           2      32955177.0  
           3     108807546.0  
           4     166272032.0  
           5      90792179.0  

[5 rows x 405 columns]